In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

Using TensorFlow backend.


In [21]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [22]:
print(x_test.shape)
print(y_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

(10000, 32, 32, 3)
(10000, 1)
50000 train samples
10000 test samples


In [23]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [24]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [25]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [26]:

x_train = x_train.astype('float32')
print(x_train[0][0])
print("====================================")
x_test = x_test.astype('float32')
x_train /= 255
print(x_train[0][0])
x_test /= 255

[[ 59.  62.  63.]
 [ 43.  46.  45.]
 [ 50.  48.  43.]
 [ 68.  54.  42.]
 [ 98.  73.  52.]
 [119.  91.  63.]
 [139. 107.  75.]
 [145. 110.  80.]
 [149. 117.  89.]
 [149. 120.  93.]
 [131. 103.  77.]
 [125.  99.  76.]
 [142. 115.  91.]
 [144. 112.  86.]
 [137. 105.  79.]
 [129.  97.  71.]
 [137. 106.  79.]
 [134. 106.  76.]
 [124.  97.  64.]
 [139. 113.  78.]
 [139. 112.  75.]
 [133. 105.  69.]
 [136. 105.  74.]
 [139. 108.  77.]
 [152. 120.  89.]
 [163. 131. 100.]
 [168. 136. 108.]
 [159. 129. 102.]
 [158. 130. 104.]
 [158. 132. 108.]
 [152. 125. 102.]
 [148. 124. 103.]]
[[0.23137255 0.24313726 0.24705882]
 [0.16862746 0.18039216 0.1764706 ]
 [0.19607843 0.1882353  0.16862746]
 [0.26666668 0.21176471 0.16470589]
 [0.38431373 0.28627452 0.20392157]
 [0.46666667 0.35686275 0.24705882]
 [0.54509807 0.41960785 0.29411766]
 [0.5686275  0.43137255 0.3137255 ]
 [0.58431375 0.45882353 0.34901962]
 [0.58431375 0.47058824 0.3647059 ]
 [0.5137255  0.40392157 0.3019608 ]
 [0.49019608 0.3882353  0.2

In [31]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
#     model.fit_generator(datagen.flow(x_train, y_train,
#                                      batch_size=batch_size),
#                         epochs=epochs,
#                         validation_data=(x_test, y_test))

Using real-time data augmentation.


In [33]:
model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,steps_per_epoch = 1000,
                        validation_data=(x_test, y_test),
                        workers=4)

Epoch 1/100
1000/1000 [==============================] - 230s 230ms/step - loss: 1.9211 - acc: 0.2911 - val_loss: 1.6709 - val_acc: 0.3989
Epoch 2/100
1000/1000 [==============================] - 215s 215ms/step - loss: 1.6694 - acc: 0.3902 - val_loss: 1.5023 - val_acc: 0.4485
Epoch 3/100
1000/1000 [==============================] - 201s 201ms/step - loss: 1.5493 - acc: 0.4335 - val_loss: 1.4389 - val_acc: 0.4890
Epoch 4/100
1000/1000 [==============================] - 205s 205ms/step - loss: 1.4781 - acc: 0.4632 - val_loss: 1.4426 - val_acc: 0.4845
Epoch 5/100
1000/1000 [==============================] - 202s 202ms/step - loss: 1.4240 - acc: 0.4844 - val_loss: 1.3362 - val_acc: 0.5197
Epoch 6/100
1000/1000 [==============================] - 189s 189ms/step - loss: 1.3705 - acc: 0.5085 - val_loss: 1.2524 - val_acc: 0.5591
Epoch 7/100
1000/1000 [==============================] - 195s 195ms/step - loss: 1.3238 - acc: 0.5243 - val_loss: 1.1898 - val_acc: 0.5748
Epoch 8/100
1000/1000 [====

KeyboardInterrupt: 

In [34]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'plt' is not defined

In [35]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at C:\Users\Karan\Desktop\cifar\saved_models\keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 11s 1ms/step
Test loss: 0.7214969015598297
Test accuracy: 0.7554


In [36]:
print(scores)

[0.7214969015598297, 0.7554]
